In [110]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [111]:
%%R
library(DESeq2)
library(magrittr)
library(SummarizedExperiment)

start_time <- Sys.time()

IN_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/input/"
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/output/"

GEO_model_training_DATA_DIR <- IN_DIR
GEO_model_training_DATA_FIL <- "rse_tcdd_data.Rdata"

ensembl2rxns.df <- read.table(paste(IN_DIR,"Ensembl2ReactomeReactions.txt",sep=""),
                              sep="\t")

load(paste(GEO_model_training_DATA_DIR,GEO_model_training_DATA_FIL,sep=""))

In [117]:
%%R
count_data <- assays(final_result)$raw_counts

count_df <- as.data.frame(count_data)
# print(colnames(colData(final_result)))
gene_id <- colData(final_result)$external_id
# print(count_df['ENSMUSG00000032310.4',])
# print(count_df['ENSMUSG00000032315.6',])
dose_data <- colData(final_result)$dose

final_table <- data.frame(
  gene_id = gene_id,
  ENSMUSG00000032310_4 = t(count_df['ENSMUSG00000032310.4',]), # 转置因为提取的是行数据
  ENSMUSG00000032315_6 = t(count_df['ENSMUSG00000032315.6',]), # 转置同上
  dose = dose_data
)
print(final_table)

        gene_id ENSMUSG00000032310.4 ENSMUSG00000032315.6  dose
V1   SRR7817611               424744                17056  0.00
V2   SRR7817612               417970                19429  0.00
V3   SRR7817614               700804                24175  0.00
V4   SRR7817615               445100                13625  0.00
V5   SRR7817616               443357                12739  0.00
V6   SRR7817617             15595098              3078910 30.00
V7   SRR7817618             15812325              3136263 30.00
V8   SRR7817619             14725193              4601435 30.00
V9   SRR7817620             14869517              4668899 30.00
V10  SRR7817621              8508590              3806013 30.00
V11  SRR7817622              8606531              3841907 30.00
V12  SRR7817623               544835                15286  0.00
V13  SRR7817624               544208                15852  0.00
V14  SRR7817625               368638                18208  0.00
V15  SRR7817626               377974    

V128 SRR1636649              2776551              1150531  1.00
V129 SRR1636650              4264645              2080798  3.00
V130 SRR1636651              4309520              2121249  3.00
V131 SRR1636652             16477482              8055606  3.00
V132 SRR1636655              8993616              4862546  3.00
V133 SRR1636656              4995818              2384848  3.00
V134 SRR1636657              5028645              2380769  3.00
V135 SRR1636588               376431                10994  0.00
V136 SRR1636589               371149                11772  0.00
V137 SRR1636590               444897                11632  0.00
V138 SRR1636600               473667                13278  0.01
V139 SRR1636609               438141                19302  0.03
V140 SRR1636610              1667577                70588  0.03
V141 SRR1636611               728466                33101  0.03
V142 SRR1636612               711029                34993  0.03
V143 SRR1636613               743675    

V256 SRR6653207               294388                11133  0.00
V257 SRR6653208               292879                11192  0.00
V258 SRR6653211               288565                18083  0.00
V259 SRR6653212               290776                15770  0.00
V260 SRR6653214              7244802              3900244 30.00
V261 SRR6653219               287725                10732  0.00
V262 SRR6653220               286919                 8721  0.00
V263 SRR6653221               303766                10985  0.00
V264 SRR6653222               298198                11021  0.00
V265 SRR6653223               339810                12976  0.00
V266 SRR6653224               338248                14930  0.00
V267 SRR6653225              6751216              3221117 30.00
V268 SRR6653226              6720181              3234113 30.00
V269 SRR6653227              7852057              3941725 30.00
V270 SRR6653229              7953175              4015639 30.00
V271 SRR6653230              7865832    

In [118]:
%%R
grep('ENSMUSG00000032315', ensembl2rxns.df$V1)

[1] 913917 913918 913919 913920 913921 913922 913923


In [5]:
%%R
print(final_result)

class: RangedSummarizedExperiment 
dim: 55421 383 
metadata(40): time_created recount3_version ... annotation recount3_url
assays(1): raw_counts
rownames(55421): ENSMUSG00000079800.2 ENSMUSG00000095092.1 ...
  ENSMUSG00000096850.1 ENSMUSG00000099871.1
rowData names(11): source type ... havana_gene tag
colnames: NULL
colData names(200): Row.names rail_id ... dose_unit gender


In [120]:
%%R

# 计算方差
variances <- apply(assay(final_result), 1, var)

# 确定方差为0的行
zero_variance_rows <- which(variances == 0)

# 如果有需要移除的行
if (length(zero_variance_rows) > 0) {
    deleted_data <- assay(final_result)[zero_variance_rows, , drop = FALSE]
    save(deleted_data, file=paste0(OUT_DIR, "deleted_data.RData"))
    # 创建新的assays和rowData
    new_assays <- assay(final_result)[-zero_variance_rows, , drop = FALSE]
    new_row_data <- rowData(final_result)[-zero_variance_rows, , drop = FALSE]

    # 使用新的数据创建一个新的SummarizedExperiment对象
    new_final_result <- SummarizedExperiment(
        assays = SimpleList(counts = new_assays),
        rowData = new_row_data,
        colData = colData(final_result)
    )
    
    # 将新对象赋值回final_result
    final_result <- new_final_result
    
    # 输出新对象的维度信息确认更新成功
    print(dim(assay(final_result)))
    print(dim(rowData(final_result)))
} else {
    cat("No rows with zero variance found.\n")
}

# 打印更新后的数据确认更改
print(head(assay(final_result)))

[1] 43252   383
[1] 43252    11
                     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
ENSMUSG00000079800.2    0    0    0    0    0    0    0    0   49    49     0
ENSMUSG00000095092.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079192.2    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079794.2    0    0    0    0    0    0    0    0    0     0    50
ENSMUSG00000094799.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000095250.1    0    0    0    0    0   50    0    0    0     0     0
                     [,12] [,13] [,14] [,15] [,16] [,17] [,18] [,19] [,20]
ENSMUSG00000079800.2    50     0    50    50     0     0     0     0     0
ENSMUSG00000095092.1     0     0     0     0     0     0     0     0     0
ENSMUSG00000079192.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000079794.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000094799.1     0     0     0     0   

ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0     50     48      0      0      0
                     [,142] [,143] [,144] [,145] [,146] [,147] [,148] [,149]
ENSMUSG00000079800.2      0      0      0      0      0      0      0      0
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,150] [,151] [,152] [,153] [,154] [,155] [,156] [,157]

ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,262] [,263] [,264] [,265] [,266] [,267] [,268] [,269]
ENSMUSG00000079800.2      0      0      0      0      0    100    150     50
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,270] [,271] [,272] [,273] [,274] [,275] [,276] [,277]
ENSMUSG00000079800.2      0      0     50      0     50     40     34     49

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [121]:
%%R
grep('ENSMUSG00000032315', rownames(assay(final_result)))

[1] 40052


In [47]:
%%R
library(plotly)
library(SummarizedExperiment)
library(htmlwidgets)
# Assuming final_result is your SummarizedExperiment object
# Extract the necessary data
dose <- as.factor(colData(final_result)$dose) # Convert dose to a factor
expression <- assay(final_result)[40052,]
SRR <- colData(final_result)$SRR
project <- colData(final_result)$project_id
# Create a data frame for plotting
plot_data <- data.frame(SRR = SRR, dose = dose, project = project, expression = expression)
# Create the Plotly scatter plot
p <- plot_ly(plot_data, x = ~dose, y = ~jitter(expression), type = 'scatter', mode = 'markers', color = ~project, colors = 'Set1', text = ~paste('SRR:', SRR, '<br>dose:', dose, '<br>project:', project), hoverinfo = 'text') %>% layout(title = 'Dose vs Expression for Gene 40051', xaxis = list(title = 'Dose'), yaxis = list(title = 'Expression'))
# Show the plot
saveWidget(p, "plotly_test.html", selfcontained = TRUE)

In [122]:
%%R
rownames(rowData(final_result))[40052]

[1] "ENSMUSG00000032315.6"


In [123]:
%%R
plot(assay(final_result)[40051,], assay(final_result)[40052,])

In [124]:
%%R
# 获取想要保留的样本的逻辑向量
keep_samples <- colData(final_result)$dose %in% c(0.00, 30.00)

# 使用这个逻辑向量来子集化 final_result
final_result <- final_result[, keep_samples]

# 打印更新后的 colData 以确认更改
print(colData(final_result))
tcdd_data <- colData(final_result)

DataFrame with 220 rows and 198 columns
    external_id   rail_id       study sra.sample_acc.x sra.experiment_acc
    <character> <integer> <character>      <character>        <character>
1    SRR7817611   2802239   SRP161461       SRS3763870         SRX4669086
2    SRR7817612   2802269   SRP161461       SRS3763870         SRX4669086
3    SRR7817614   2802332   SRP161461       SRS3763872         SRX4669087
4    SRR7817615   2802363   SRP161461       SRS3763868         SRX4669088
5    SRR7817616   2802395   SRP161461       SRS3763868         SRX4669088
...         ...       ...         ...              ...                ...
216  SRR4317656    930487   SRP090688       SRS1721112         SRX2202381
217  SRR4317657    930495   SRP090688       SRS1721113         SRX2202382
218  SRR4317658    930503   SRP090688       SRS1721113         SRX2202382
219  SRR4317659    930510   SRP090688       SRS1721115         SRX2202383
220  SRR4317660    930566   SRP090688       SRS1721115         SRX220238

           <integer>         <character> <integer>           <integer>
1           14559305 2019-05-08 16:15:15 266190129           727965250
2           14821846 2019-05-08 16:15:15 272135592           741092300
3           20958607 2019-05-08 16:15:15 386681562          1047930350
4           14649783 2019-05-08 16:15:15 270011270           732489150
5           14712677 2019-05-08 16:15:15 272283276           735633850
...              ...                 ...       ...                 ...
216         22185618 2017-02-28 15:45:25 654904457          1109280900
217         14055416 2017-02-28 15:45:25 421586591           702770800
218         13639614 2017-02-28 15:45:25 405226312           681980700
219         18673575 2017-02-28 15:45:25 552543054           933678750
220         18752417 2017-02-28 15:45:25 555840036           937620850
    sra.run_total_spots sra.num_reads sra.num_spots          sra.read_info
              <integer>     <integer>     <integer>            <character

                                         <integer>               <numeric>
1                                        565196732                   99.80
2                                        575058122                   99.81
3                                        827957213                   99.64
4                                        588026949                   99.76
5                                        590030308                   99.74
...                                            ...                     ...
216                                      859611848                   98.27
217                                      550633748                   98.26
218                                      534747298                   98.28
219                                      730839638                   98.55
220                                      734131403                   98.56
    recount_qc.bc_auc.unique_% recount_qc.bc_frag.count
                     <numeric>              

                                <integer>                             <integer>
1                                12340743                              18553206
2                                12554723                              18888302
3                                17997081                              26657641
4                                12603672                              18461698
5                                12650973                              18536386
...                                   ...                                   ...
216                              18353927                              28754671
217                              11613299                              18024636
218                              11275394                              17481677
219                              15757135                              24057569
220                              15828817                              24166886
    recount_qc.gene_fc_count_unique.assi

                                                    <integer>
1                                                           0
2                                                           0
3                                                           0
4                                                           0
5                                                           0
...                                                       ...
216                                                         0
217                                                         0
218                                                         0
219                                                         0
220                                                         0
    recount_qc.star.%_of_reads_unmapped:_too_short
                                         <numeric>
1                                             1.28
2                                             1.30
3                                             1.04
4

    recount_qc.star.insertion_average_length2
                                    <integer>
1                                           0
2                                           0
3                                           0
4                                           0
5                                           0
...                                       ...
216                                         0
217                                         0
218                                         0
219                                         0
220                                         0
    recount_qc.star.insertion_rate_per_base
                                  <numeric>
1                                         0
2                                         0
3                                         0
4                                         0
5                                         0
...                                     ...
216                                       0
217   

217                                                        0
218                                                        0
219                                                        0
220                                                        0
    recount_qc.star.number_of_reads_mapped_to_too_many_loci
                                                  <integer>
1                                                    619021
2                                                    633821
3                                                    605238
4                                                    367160
5                                                    370518
...                                                     ...
216                                                  357246
217                                                  234346
218                                                  228740
219                                                  281720
220                                 

219                                                   0
220                                                   0
    recount_qc.star.number_of_splices:_annotated_(sjdb)2
                                               <integer>
1                                                      0
2                                                      0
3                                                      0
4                                                      0
5                                                      0
...                                                  ...
216                                                    0
217                                                    0
218                                                    0
219                                                    0
220                                                    0
    recount_qc.star.number_of_splices:_gc/ag
                                   <integer>
1                                       5807
2           

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [125]:
%%R
# tcdd_data <- colData(final_result)

dose_counts <- table(tcdd_data$dose)

print(dose_counts)

dose_counts_df <- as.data.frame(dose_counts)

colnames(dose_counts_df) <- c("Dose", "Count")

write.csv(dose_counts_df, paste0(OUT_DIR, "dose_counts030.csv"), row.names=FALSE)
# tissues_to_remove <- names(tissue_counts[tissue_counts <= 20])

# rse_gene_80 <- rse_gene_80[, !(tissue_data$Major_tissue %in% c(tissues_to_remove, 'Spleen'))]

# new_tissue_counts <- table(colData(rse_gene_80)$Major_tissue)
# print(new_tissue_counts)

# row_totals <- rowSums(assays(rse_gene_80)$raw_counts)

# rse_gene_80 <- rse_gene_80[row_totals > 0, ]


  0  30 
110 110 


In [126]:
%%R
combined_gender <- ifelse(tcdd_data$Sex != "", tcdd_data$Sex, tcdd_data$gender)
print(combined_gender)

  [1] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
  [9] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
 [17] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
 [25] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
 [33] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
 [41] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
 [49] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
 [57] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
 [65] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
 [73] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
 [81] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
 [89] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
 [97] "female" "female" "female" "female" "female" "female" "fem

In [128]:
%%R
write.table(tcdd_data$project_id,file=paste(OUT_DIR,"tcdd_project_id030.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

In [129]:
%%R
write.table(combined_gender,file=paste(OUT_DIR,"tcdd_gender030.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

In [132]:
%%R
write.table(tcdd_data$external_id,file=paste(OUT_DIR,"tcdd_sample_id030.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

In [136]:
%%R
write.table(tcdd_data$dose,file=paste(OUT_DIR,"tcdd_dose030.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

In [137]:
%%R
saveRDS(tcdd_data$dose,file=paste(OUT_DIR,"tcdd_dose_detail_vec030.Rds",sep=""))

In [138]:
%%R
saveRDS(tcdd_data$external_id,file=paste(OUT_DIR,"tcdd_sample_detail_vec030.Rds",sep=""))

In [139]:
%%R
tcdd.df <- final_result %>% SummarizedExperiment::assay() %>% as.data.frame()
print(tcdd.df)

                           V1      V2       V3      V4      V5       V6
ENSMUSG00000079800.2        0       0        0       0       0        0
ENSMUSG00000095092.1        0       0        0       0       0        0
ENSMUSG00000079192.2        0       0        0       0       0        0
ENSMUSG00000079794.2        0       0        0       0       0        0
ENSMUSG00000094799.1        0       0        0       0       0        0
ENSMUSG00000095250.1        0       0        0       0       0       50
ENSMUSG00000095787.1        0       0        0       0       0        0
ENSMUSG00000096100.1        0       0        0       0       0        0
ENSMUSG00000094054.1        0       0        0       0       0        0
ENSMUSG00000095672.1        0       0        0       0       0        0
ENSMUSG00000079190.3        0      50        0       0     150       34
ENSMUSG00000094514.1       49       0        0      96       0       49
ENSMUSG00000094915.1        0       0        0       0       0  

ENSMUSG00000103884.1        0       0        0       0       0        0
ENSMUSG00000084353.1     4057    5357     4781    2862    2448     5508
ENSMUSG00000103933.1    31020   29109    46442   34434   34721    43708
ENSMUSG00000076135.1        0       8        0       0       0        0
ENSMUSG00000086195.1       36      68       20       0       0       34
ENSMUSG00000104504.1       31       6        0      12      18        0
ENSMUSG00000066693.2     3448    3942     5114    3616    4694     5398
ENSMUSG00000102316.1        0       0        0       0       0        0
ENSMUSG00000103819.1        0       0        0       0       0        0
ENSMUSG00000102871.1        0       0        0       0       0        0
ENSMUSG00000025909.16     100     100       50     100     193      335
ENSMUSG00000101571.2        9     350      300     250     300      200
ENSMUSG00000104428.1        0       0        0       0       0        0
ENSMUSG00000102272.1     1940    2143     2572    2435    2839  

ENSMUSG00000091020.3     2844    3042     3237    3246    2149      996
ENSMUSG00000101640.1       54       0        0       0       0        0
ENSMUSG00000042686.5      650     600      300     150     448       50
ENSMUSG00000025777.8        0       0        0       0       0        0
ENSMUSG00000099899.1        0       0        0       0       0        0
ENSMUSG00000100398.1      150     150      640     398     297      546
ENSMUSG00000101589.1     1540    2477     3325    3539    2535     4773
ENSMUSG00000102048.1      150     349      300      50     150     1193
ENSMUSG00000067780.3       50      50      250       0       0      199
ENSMUSG00000100053.1       50       0       66       0       0        0
ENSMUSG00000085125.7        0       0        0       0       0        0
ENSMUSG00000025776.13       0       0        0       0       0        0
ENSMUSG00000099895.1        0       0        0       0       0        0
ENSMUSG00000100554.1     1299     995     1240    1230    1199  

ENSMUSG00000080391.1        0       0        0       0       0        0
ENSMUSG00000099492.1    16060   14989    20567   14874   15958    38645
ENSMUSG00000101726.1        0       0        0       0       0        0
ENSMUSG00000100261.1     4010    2899     4522    3162    3151     3954
ENSMUSG00000101180.1        0       0        0       0       0       10
ENSMUSG00000103783.1       50      50        0      50       0        0
ENSMUSG00000104315.1        0       0        0       0       0        0
ENSMUSG00000048874.15   24960   27002    36666   26259   30458    15312
ENSMUSG00000026064.16   26156   27717    52353   37205   36393    27066
ENSMUSG00000117310.2    25741   27418    50896   36651   35701    27066
ENSMUSG00000117091.1        0       0        0       0       0        0
ENSMUSG00000086726.3        0       0        0       0       0        0
ENSMUSG00000100237.1        0      50        0      35       0        0
ENSMUSG00000100910.1        0       0        0       0       0  

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

        0        0
ENSMUSG00000103708.1        0        0       0       0        0        0
ENSMUSG00000102434.1        0        0       0       0        0        0
ENSMUSG00000104523.1     1602     6632    9087    8916     7489     8175
ENSMUSG00000045216.7    18169    49626   43894   42839    41840    41974
ENSMUSG00000102202.1      150      696     347     807      300      100
ENSMUSG00000098752.1       33        8       2       2        8       12
ENSMUSG00000037470.14   28957    75061   31702   35693    30938    29758
ENSMUSG00000047180.8    11214    25764    3983    3891     1844     2210
ENSMUSG00000037447.16    2281     6680    1238    1142     1305     1029
ENSMUSG00000102762.1        0        0      22       0       49        0
ENSMUSG00000098680.1        7        2      42       0        6        6
ENSMUSG00000010453.12   17977    51016   26780   28767    26326    28236
ENSMUSG00000037432.15     432      755     685     877      934      647
ENSMUSG00000001143.13    2964   

ENSMUSG00000025903.14   135471   135105   131718   124559    83899   84599
ENSMUSG00000062588.4       887      899     3784     2943     1733    1790
ENSMUSG00000104217.1     33010    34008    30993    31958    21858   25065
ENSMUSG00000033813.15    13957    12415    25833    29755    15261   15134
ENSMUSG00000103280.1        50      100       50       50      100      50
ENSMUSG00000091305.1         0        0       94       94       94      47
ENSMUSG00000102653.1         0        0        0        0        0       0
ENSMUSG00000085623.1         0        0        0        0        0       0
ENSMUSG00000091665.1         0        0        0        0        0       0
ENSMUSG00000002459.17       50        0       49      100        0      50
ENSMUSG00000033793.12    32309    31564   159735   158378    79273   74552
ENSMUSG00000104352.1         0        0        0        0        0       0
ENSMUSG00000104046.1         0        0        0       48        0       0
ENSMUSG00000025905.14    

ENSMUSG00000103085.1       150      100       50       50       50      50
ENSMUSG00000102664.1        50       50        0        0        0       0
ENSMUSG00000005886.14    27123    28023    57287    56796    34347   31994
ENSMUSG00000101476.1         0        0        0        0        0      50
ENSMUSG00000101171.1      1097     1049     2443     2147     1845    1999
ENSMUSG00000025935.10    83717    79299   160490   152329    84508   82191
ENSMUSG00000102982.1       950     1329      896     1447      896    1128
ENSMUSG00000025937.6    126948   133860   145420   139239   109881  101609
ENSMUSG00000067813.3     11198    11753    41212    35252    27024   26474
ENSMUSG00000097711.1       113        0      698      650      200     190
ENSMUSG00000089358.1         0        0        0        0        0       0
ENSMUSG00000104209.1         0        0        0        0        0       0
ENSMUSG00000025932.14        0        0      100        0        0       0
ENSMUSG00000025930.6     

ENSMUSG00000102766.1         0        0        0        0        0       0
ENSMUSG00000065405.3         0        0        0        0        0       0
ENSMUSG00000065567.1         0        0        0        0        0       0
ENSMUSG00000097970.1        13       89       62       54       34     187
ENSMUSG00000100775.1        56      250      700      500      100     599
ENSMUSG00000026158.11     6121     6638    15817    13193     6091    5517
ENSMUSG00000097920.1        50        0      193      498      343      50
ENSMUSG00000026156.8     10069     8695    34511    31734    15357   12567
ENSMUSG00000026155.13    13718    12319    47651    45586    20783   18310
ENSMUSG00000026154.4      9746     7875    13694    15236     5988    8245
ENSMUSG00000102495.1         0       63       34      163       62       0
ENSMUSG00000026153.15    18465    18261    65724    66697    30640   28461
ENSMUSG00000102398.1         0        0       50       50      250     150
ENSMUSG00000097670.1     

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [140]:
%%R
grep('ENSMUSG00000032315', ensembl_wo_ids)

[1] 40052


In [141]:
%%R
ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(tcdd.df))
# print(ensembl_wo_ids)

In [142]:
%%R
deleted_ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(deleted_data))
print(deleted_ensembl_wo_ids)

    [1] "ENSMUSG00000096776" "ENSMUSG00000096244" "ENSMUSG00000095450"
    [4] "ENSMUSG00000089163" "ENSMUSG00000099278" "ENSMUSG00000094121"
    [7] "ENSMUSG00000079764" "ENSMUSG00000079774" "ENSMUSG00000094741"
   [10] "ENSMUSG00000074720" "ENSMUSG00000094337" "ENSMUSG00000095570"
   [13] "ENSMUSG00000098647" "ENSMUSG00000094431" "ENSMUSG00000095247"
   [16] "ENSMUSG00000095585" "ENSMUSG00000096873" "ENSMUSG00000095755"
   [19] "ENSMUSG00000095207" "ENSMUSG00000093828" "ENSMUSG00000095623"
   [22] "ENSMUSG00000095666" "ENSMUSG00000096680" "ENSMUSG00000096236"
   [25] "ENSMUSG00000096756" "ENSMUSG00000094722" "ENSMUSG00000095728"
   [28] "ENSMUSG00000094836" "ENSMUSG00000094383" "ENSMUSG00000094474"
   [31] "ENSMUSG00000096550" "ENSMUSG00000094172" "ENSMUSG00000091585"
   [34] "ENSMUSG00000095763" "ENSMUSG00000095523" "ENSMUSG00000095475"
   [37] "ENSMUSG00000095019" "ENSMUSG00000102693" "ENSMUSG00000064842"
   [40] "ENSMUSG00000102851" "ENSMUSG00000103377" "ENSMUSG00000102348"
   [43

  [346] "ENSMUSG00000104104" "ENSMUSG00000087883" "ENSMUSG00000099892"
  [349] "ENSMUSG00000100977" "ENSMUSG00000077039" "ENSMUSG00000100684"
  [352] "ENSMUSG00000099770" "ENSMUSG00000088510" "ENSMUSG00000101890"
  [355] "ENSMUSG00000087908" "ENSMUSG00000088594" "ENSMUSG00000100955"
  [358] "ENSMUSG00000099469" "ENSMUSG00000101297" "ENSMUSG00000100059"
  [361] "ENSMUSG00000088161" "ENSMUSG00000100935" "ENSMUSG00000103991"
  [364] "ENSMUSG00000084704" "ENSMUSG00000103060" "ENSMUSG00000089866"
  [367] "ENSMUSG00000089206" "ENSMUSG00000084228" "ENSMUSG00000044594"
  [370] "ENSMUSG00000058017" "ENSMUSG00000080729" "ENSMUSG00000073602"
  [373] "ENSMUSG00000073601" "ENSMUSG00000087210" "ENSMUSG00000089278"
  [376] "ENSMUSG00000077237" "ENSMUSG00000099484" "ENSMUSG00000104478"
  [379] "ENSMUSG00000101231" "ENSMUSG00000100884" "ENSMUSG00000089037"
  [382] "ENSMUSG00000084741" "ENSMUSG00000100290" "ENSMUSG00000100758"
  [385] "ENSMUSG00000102728" "ENSMUSG00000088691" "ENSMUSG00000099884"
  [388

  [691] "ENSMUSG00000077749" "ENSMUSG00000111753" "ENSMUSG00000088423"
  [694] "ENSMUSG00000044499" "ENSMUSG00000103299" "ENSMUSG00000111549"
  [697] "ENSMUSG00000096080" "ENSMUSG00000077576" "ENSMUSG00000111234"
  [700] "ENSMUSG00000088925" "ENSMUSG00000111048" "ENSMUSG00000111208"
  [703] "ENSMUSG00000110782" "ENSMUSG00000094521" "ENSMUSG00000094311"
  [706] "ENSMUSG00000111183" "ENSMUSG00000019834" "ENSMUSG00000111098"
  [709] "ENSMUSG00000110889" "ENSMUSG00000046922" "ENSMUSG00000111576"
  [712] "ENSMUSG00000094663" "ENSMUSG00000111315" "ENSMUSG00000093298"
  [715] "ENSMUSG00000087891" "ENSMUSG00000099893" "ENSMUSG00000098642"
  [718] "ENSMUSG00000076282" "ENSMUSG00000110891" "ENSMUSG00000077082"
  [721] "ENSMUSG00000098418" "ENSMUSG00000077591" "ENSMUSG00000098993"
  [724] "ENSMUSG00000098409" "ENSMUSG00000111182" "ENSMUSG00000094681"
  [727] "ENSMUSG00000065718" "ENSMUSG00000111499" "ENSMUSG00000111419"
  [730] "ENSMUSG00000110843" "ENSMUSG00000111719" "ENSMUSG00000112062"
  [733

 [1036] "ENSMUSG00000107400" "ENSMUSG00000094734" "ENSMUSG00000108114"
 [1039] "ENSMUSG00000058513" "ENSMUSG00000088147" "ENSMUSG00000096858"
 [1042] "ENSMUSG00000094702" "ENSMUSG00000071065" "ENSMUSG00000050478"
 [1045] "ENSMUSG00000069421" "ENSMUSG00000063173" "ENSMUSG00000049052"
 [1048] "ENSMUSG00000052818" "ENSMUSG00000061961" "ENSMUSG00000107399"
 [1051] "ENSMUSG00000058071" "ENSMUSG00000094295" "ENSMUSG00000095608"
 [1054] "ENSMUSG00000107423" "ENSMUSG00000048745" "ENSMUSG00000044897"
 [1057] "ENSMUSG00000108253" "ENSMUSG00000095804" "ENSMUSG00000112688"
 [1060] "ENSMUSG00000059862" "ENSMUSG00000045559" "ENSMUSG00000088352"
 [1063] "ENSMUSG00000085559" "ENSMUSG00000085512" "ENSMUSG00000064632"
 [1066] "ENSMUSG00000077304" "ENSMUSG00000088688" "ENSMUSG00000080833"
 [1069] "ENSMUSG00000086808" "ENSMUSG00000082897" "ENSMUSG00000098853"
 [1072] "ENSMUSG00000084318" "ENSMUSG00000089247" "ENSMUSG00000104137"
 [1075] "ENSMUSG00000088861" "ENSMUSG00000064766" "ENSMUSG00000087990"
 [1078

 [1381] "ENSMUSG00000086617" "ENSMUSG00000084200" "ENSMUSG00000064788"
 [1384] "ENSMUSG00000000125" "ENSMUSG00000080694" "ENSMUSG00000089479"
 [1387] "ENSMUSG00000086089" "ENSMUSG00000087184" "ENSMUSG00000088279"
 [1390] "ENSMUSG00000098989" "ENSMUSG00000064691" "ENSMUSG00000087139"
 [1393] "ENSMUSG00000085324" "ENSMUSG00000020624" "ENSMUSG00000086760"
 [1396] "ENSMUSG00000086135" "ENSMUSG00000078600" "ENSMUSG00000075431"
 [1399] "ENSMUSG00000089460" "ENSMUSG00000084130" "ENSMUSG00000064588"
 [1402] "ENSMUSG00000081318" "ENSMUSG00000078586" "ENSMUSG00000086588"
 [1405] "ENSMUSG00000085336" "ENSMUSG00000110653" "ENSMUSG00000082562"
 [1408] "ENSMUSG00000085059" "ENSMUSG00000084887" "ENSMUSG00000080616"
 [1411] "ENSMUSG00000083155" "ENSMUSG00000099690" "ENSMUSG00000085970"
 [1414] "ENSMUSG00000112240" "ENSMUSG00000092168" "ENSMUSG00000112723"
 [1417] "ENSMUSG00000112559" "ENSMUSG00000088209" "ENSMUSG00000102096"
 [1420] "ENSMUSG00000099360" "ENSMUSG00000077259" "ENSMUSG00000112787"
 [1423

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "ENSMUSG00000102966" "ENSMUSG00000104425"
[11008] "ENSMUSG00000102146" "ENSMUSG00000102159" "ENSMUSG00000102506"
[11011] "ENSMUSG00000102826" "ENSMUSG00000102230" "ENSMUSG00000104345"
[11014] "ENSMUSG00000102262" "ENSMUSG00000103528" "ENSMUSG00000102772"
[11017] "ENSMUSG00000103425" "ENSMUSG00000102486" "ENSMUSG00000103250"
[11020] "ENSMUSG00000103077" "ENSMUSG00000103787" "ENSMUSG00000103987"
[11023] "ENSMUSG00000103063" "ENSMUSG00000102957" "ENSMUSG00000103269"
[11026] "ENSMUSG00000103954" "ENSMUSG00000103894" "ENSMUSG00000103795"
[11029] "ENSMUSG00000103919" "ENSMUSG00000103812" "ENSMUSG00000104259"
[11032] "ENSMUSG00000102487" "ENSMUSG00000102847" "ENSMUSG00000103773"
[11035] "ENSMUSG00000102347" "ENSMUSG00000104480" "ENSMUSG00000104062"
[11038] "ENSMUSG00000103231" "ENSMUSG00000102318" "ENSMUSG00000103074"
[11041] "ENSMUSG00000102965" "ENSMUSG00000104289" "ENSMUSG00000102730"
[11044] "ENSMUSG00000102529" "ENSMUSG00000103589" "ENSMUSG00000102396"
[11047] "ENSMUSG00000104255" "ENSM

[11350] "ENSMUSG00000100644" "ENSMUSG00000098868" "ENSMUSG00000099228"
[11353] "ENSMUSG00000099379" "ENSMUSG00000100070" "ENSMUSG00000096885"
[11356] "ENSMUSG00000095452" "ENSMUSG00000100797" "ENSMUSG00000100649"
[11359] "ENSMUSG00000100196" "ENSMUSG00000099628" "ENSMUSG00000096120"
[11362] "ENSMUSG00000099897" "ENSMUSG00000099785" "ENSMUSG00000094071"
[11365] "ENSMUSG00000101536" "ENSMUSG00000099652" "ENSMUSG00000100737"
[11368] "ENSMUSG00000099780" "ENSMUSG00000099849" "ENSMUSG00000100822"
[11371] "ENSMUSG00000101471" "ENSMUSG00000101455" "ENSMUSG00000099700"
[11374] "ENSMUSG00000101811" "ENSMUSG00000099833" "ENSMUSG00000101247"
[11377] "ENSMUSG00000099408" "ENSMUSG00000099542" "ENSMUSG00000100107"
[11380] "ENSMUSG00000100051" "ENSMUSG00000099977" "ENSMUSG00000099415"
[11383] "ENSMUSG00000096332" "ENSMUSG00000100693" "ENSMUSG00000099638"
[11386] "ENSMUSG00000094181" "ENSMUSG00000102081" "ENSMUSG00000102031"
[11389] "ENSMUSG00000100408" "ENSMUSG00000100991" "ENSMUSG00000101649"
[11392

[11695] "ENSMUSG00000101790" "ENSMUSG00000100452" "ENSMUSG00000101403"
[11698] "ENSMUSG00000101830" "ENSMUSG00000100322" "ENSMUSG00000099350"
[11701] "ENSMUSG00000100329" "ENSMUSG00000099940" "ENSMUSG00000101766"
[11704] "ENSMUSG00000101530" "ENSMUSG00000100676" "ENSMUSG00000101539"
[11707] "ENSMUSG00000100353" "ENSMUSG00000101022" "ENSMUSG00000100165"
[11710] "ENSMUSG00000100077" "ENSMUSG00000101569" "ENSMUSG00000101184"
[11713] "ENSMUSG00000099386" "ENSMUSG00000095103" "ENSMUSG00000095785"
[11716] "ENSMUSG00000101982" "ENSMUSG00000099487" "ENSMUSG00000101507"
[11719] "ENSMUSG00000093993" "ENSMUSG00000100956" "ENSMUSG00000100942"
[11722] "ENSMUSG00000100333" "ENSMUSG00000100126" "ENSMUSG00000101679"
[11725] "ENSMUSG00000099434" "ENSMUSG00000101177" "ENSMUSG00000100993"
[11728] "ENSMUSG00000099778" "ENSMUSG00000101512" "ENSMUSG00000100136"
[11731] "ENSMUSG00000101256" "ENSMUSG00000101758" "ENSMUSG00000099395"
[11734] "ENSMUSG00000100982" "ENSMUSG00000100988" "ENSMUSG00000099368"
[11737

[12040] "ENSMUSG00000101273" "ENSMUSG00000101508" "ENSMUSG00000100339"
[12043] "ENSMUSG00000099546" "ENSMUSG00000101884" "ENSMUSG00000100906"
[12046] "ENSMUSG00000100092" "ENSMUSG00000100284" "ENSMUSG00000096523"
[12049] "ENSMUSG00000100986" "ENSMUSG00000101718" "ENSMUSG00000094454"
[12052] "ENSMUSG00000099736" "ENSMUSG00000101657" "ENSMUSG00000100740"
[12055] "ENSMUSG00000100718" "ENSMUSG00000100407" "ENSMUSG00000100946"
[12058] "ENSMUSG00000101851" "ENSMUSG00000100531" "ENSMUSG00000099483"
[12061] "ENSMUSG00000101742" "ENSMUSG00000100024" "ENSMUSG00000094660"
[12064] "ENSMUSG00000101052" "ENSMUSG00000100881" "ENSMUSG00000101160"
[12067] "ENSMUSG00000099840" "ENSMUSG00000100951" "ENSMUSG00000101068"
[12070] "ENSMUSG00000101405" "ENSMUSG00000099633" "ENSMUSG00000102079"
[12073] "ENSMUSG00000100449" "ENSMUSG00000101990" "ENSMUSG00000099642"
[12076] "ENSMUSG00000101354" "ENSMUSG00000101835" "ENSMUSG00000102127"
[12079] "ENSMUSG00000099362" "ENSMUSG00000099921" "ENSMUSG00000095705"
[12082

In [145]:
%%R
rownames(tcdd.df) <- ensembl_wo_ids
# print(rownames(tcdd.df))

In [146]:
%%R
rownames(deleted_data) <- deleted_ensembl_wo_ids
print(rownames(deleted_data))

    [1] "ENSMUSG00000096776" "ENSMUSG00000096244" "ENSMUSG00000095450"
    [4] "ENSMUSG00000089163" "ENSMUSG00000099278" "ENSMUSG00000094121"
    [7] "ENSMUSG00000079764" "ENSMUSG00000079774" "ENSMUSG00000094741"
   [10] "ENSMUSG00000074720" "ENSMUSG00000094337" "ENSMUSG00000095570"
   [13] "ENSMUSG00000098647" "ENSMUSG00000094431" "ENSMUSG00000095247"
   [16] "ENSMUSG00000095585" "ENSMUSG00000096873" "ENSMUSG00000095755"
   [19] "ENSMUSG00000095207" "ENSMUSG00000093828" "ENSMUSG00000095623"
   [22] "ENSMUSG00000095666" "ENSMUSG00000096680" "ENSMUSG00000096236"
   [25] "ENSMUSG00000096756" "ENSMUSG00000094722" "ENSMUSG00000095728"
   [28] "ENSMUSG00000094836" "ENSMUSG00000094383" "ENSMUSG00000094474"
   [31] "ENSMUSG00000096550" "ENSMUSG00000094172" "ENSMUSG00000091585"
   [34] "ENSMUSG00000095763" "ENSMUSG00000095523" "ENSMUSG00000095475"
   [37] "ENSMUSG00000095019" "ENSMUSG00000102693" "ENSMUSG00000064842"
   [40] "ENSMUSG00000102851" "ENSMUSG00000103377" "ENSMUSG00000102348"
   [43

  [346] "ENSMUSG00000104104" "ENSMUSG00000087883" "ENSMUSG00000099892"
  [349] "ENSMUSG00000100977" "ENSMUSG00000077039" "ENSMUSG00000100684"
  [352] "ENSMUSG00000099770" "ENSMUSG00000088510" "ENSMUSG00000101890"
  [355] "ENSMUSG00000087908" "ENSMUSG00000088594" "ENSMUSG00000100955"
  [358] "ENSMUSG00000099469" "ENSMUSG00000101297" "ENSMUSG00000100059"
  [361] "ENSMUSG00000088161" "ENSMUSG00000100935" "ENSMUSG00000103991"
  [364] "ENSMUSG00000084704" "ENSMUSG00000103060" "ENSMUSG00000089866"
  [367] "ENSMUSG00000089206" "ENSMUSG00000084228" "ENSMUSG00000044594"
  [370] "ENSMUSG00000058017" "ENSMUSG00000080729" "ENSMUSG00000073602"
  [373] "ENSMUSG00000073601" "ENSMUSG00000087210" "ENSMUSG00000089278"
  [376] "ENSMUSG00000077237" "ENSMUSG00000099484" "ENSMUSG00000104478"
  [379] "ENSMUSG00000101231" "ENSMUSG00000100884" "ENSMUSG00000089037"
  [382] "ENSMUSG00000084741" "ENSMUSG00000100290" "ENSMUSG00000100758"
  [385] "ENSMUSG00000102728" "ENSMUSG00000088691" "ENSMUSG00000099884"
  [388

  [691] "ENSMUSG00000077749" "ENSMUSG00000111753" "ENSMUSG00000088423"
  [694] "ENSMUSG00000044499" "ENSMUSG00000103299" "ENSMUSG00000111549"
  [697] "ENSMUSG00000096080" "ENSMUSG00000077576" "ENSMUSG00000111234"
  [700] "ENSMUSG00000088925" "ENSMUSG00000111048" "ENSMUSG00000111208"
  [703] "ENSMUSG00000110782" "ENSMUSG00000094521" "ENSMUSG00000094311"
  [706] "ENSMUSG00000111183" "ENSMUSG00000019834" "ENSMUSG00000111098"
  [709] "ENSMUSG00000110889" "ENSMUSG00000046922" "ENSMUSG00000111576"
  [712] "ENSMUSG00000094663" "ENSMUSG00000111315" "ENSMUSG00000093298"
  [715] "ENSMUSG00000087891" "ENSMUSG00000099893" "ENSMUSG00000098642"
  [718] "ENSMUSG00000076282" "ENSMUSG00000110891" "ENSMUSG00000077082"
  [721] "ENSMUSG00000098418" "ENSMUSG00000077591" "ENSMUSG00000098993"
  [724] "ENSMUSG00000098409" "ENSMUSG00000111182" "ENSMUSG00000094681"
  [727] "ENSMUSG00000065718" "ENSMUSG00000111499" "ENSMUSG00000111419"
  [730] "ENSMUSG00000110843" "ENSMUSG00000111719" "ENSMUSG00000112062"
  [733

 [1036] "ENSMUSG00000107400" "ENSMUSG00000094734" "ENSMUSG00000108114"
 [1039] "ENSMUSG00000058513" "ENSMUSG00000088147" "ENSMUSG00000096858"
 [1042] "ENSMUSG00000094702" "ENSMUSG00000071065" "ENSMUSG00000050478"
 [1045] "ENSMUSG00000069421" "ENSMUSG00000063173" "ENSMUSG00000049052"
 [1048] "ENSMUSG00000052818" "ENSMUSG00000061961" "ENSMUSG00000107399"
 [1051] "ENSMUSG00000058071" "ENSMUSG00000094295" "ENSMUSG00000095608"
 [1054] "ENSMUSG00000107423" "ENSMUSG00000048745" "ENSMUSG00000044897"
 [1057] "ENSMUSG00000108253" "ENSMUSG00000095804" "ENSMUSG00000112688"
 [1060] "ENSMUSG00000059862" "ENSMUSG00000045559" "ENSMUSG00000088352"
 [1063] "ENSMUSG00000085559" "ENSMUSG00000085512" "ENSMUSG00000064632"
 [1066] "ENSMUSG00000077304" "ENSMUSG00000088688" "ENSMUSG00000080833"
 [1069] "ENSMUSG00000086808" "ENSMUSG00000082897" "ENSMUSG00000098853"
 [1072] "ENSMUSG00000084318" "ENSMUSG00000089247" "ENSMUSG00000104137"
 [1075] "ENSMUSG00000088861" "ENSMUSG00000064766" "ENSMUSG00000087990"
 [1078

 [1381] "ENSMUSG00000086617" "ENSMUSG00000084200" "ENSMUSG00000064788"
 [1384] "ENSMUSG00000000125" "ENSMUSG00000080694" "ENSMUSG00000089479"
 [1387] "ENSMUSG00000086089" "ENSMUSG00000087184" "ENSMUSG00000088279"
 [1390] "ENSMUSG00000098989" "ENSMUSG00000064691" "ENSMUSG00000087139"
 [1393] "ENSMUSG00000085324" "ENSMUSG00000020624" "ENSMUSG00000086760"
 [1396] "ENSMUSG00000086135" "ENSMUSG00000078600" "ENSMUSG00000075431"
 [1399] "ENSMUSG00000089460" "ENSMUSG00000084130" "ENSMUSG00000064588"
 [1402] "ENSMUSG00000081318" "ENSMUSG00000078586" "ENSMUSG00000086588"
 [1405] "ENSMUSG00000085336" "ENSMUSG00000110653" "ENSMUSG00000082562"
 [1408] "ENSMUSG00000085059" "ENSMUSG00000084887" "ENSMUSG00000080616"
 [1411] "ENSMUSG00000083155" "ENSMUSG00000099690" "ENSMUSG00000085970"
 [1414] "ENSMUSG00000112240" "ENSMUSG00000092168" "ENSMUSG00000112723"
 [1417] "ENSMUSG00000112559" "ENSMUSG00000088209" "ENSMUSG00000102096"
 [1420] "ENSMUSG00000099360" "ENSMUSG00000077259" "ENSMUSG00000112787"
 [1423

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[10996] "ENSMUSG00000102255" "ENSMUSG00000103365" "ENSMUSG00000103829"
[10999] "ENSMUSG00000102668" "ENSMUSG00000104131" "ENSMUSG00000103023"
[11002] "ENSMUSG00000103689" "ENSMUSG00000094294" "ENSMUSG00000093848"
[11005] "ENSMUSG00000102429" "ENSMUSG00000102966" "ENSMUSG00000104425"
[11008] "ENSMUSG00000102146" "ENSMUSG00000102159" "ENSMUSG00000102506"
[11011] "ENSMUSG00000102826" "ENSMUSG00000102230" "ENSMUSG00000104345"
[11014] "ENSMUSG00000102262" "ENSMUSG00000103528" "ENSMUSG00000102772"
[11017] "ENSMUSG00000103425" "ENSMUSG00000102486" "ENSMUSG00000103250"
[11020] "ENSMUSG00000103077" "ENSMUSG00000103787" "ENSMUSG00000103987"
[11023] "ENSMUSG00000103063" "ENSMUSG00000102957" "ENSMUSG00000103269"
[11026] "ENSMUSG00000103954" "ENSMUSG00000103894" "ENSMUSG00000103795"
[11029] "ENSMUSG00000103919" "ENSMUSG00000103812" "ENSMUSG00000104259"
[11032] "ENSMUSG00000102487" "ENSMUSG00000102847" "ENSMUSG00000103773"
[11035] "ENSMUSG00000102347" "ENSMUSG00000104480" "ENSMUSG00000104062"
[11038

[11341] "ENSMUSG00000099786" "ENSMUSG00000100043" "ENSMUSG00000094354"
[11344] "ENSMUSG00000095725" "ENSMUSG00000100900" "ENSMUSG00000095433"
[11347] "ENSMUSG00000100508" "ENSMUSG00000100771" "ENSMUSG00000101090"
[11350] "ENSMUSG00000100644" "ENSMUSG00000098868" "ENSMUSG00000099228"
[11353] "ENSMUSG00000099379" "ENSMUSG00000100070" "ENSMUSG00000096885"
[11356] "ENSMUSG00000095452" "ENSMUSG00000100797" "ENSMUSG00000100649"
[11359] "ENSMUSG00000100196" "ENSMUSG00000099628" "ENSMUSG00000096120"
[11362] "ENSMUSG00000099897" "ENSMUSG00000099785" "ENSMUSG00000094071"
[11365] "ENSMUSG00000101536" "ENSMUSG00000099652" "ENSMUSG00000100737"
[11368] "ENSMUSG00000099780" "ENSMUSG00000099849" "ENSMUSG00000100822"
[11371] "ENSMUSG00000101471" "ENSMUSG00000101455" "ENSMUSG00000099700"
[11374] "ENSMUSG00000101811" "ENSMUSG00000099833" "ENSMUSG00000101247"
[11377] "ENSMUSG00000099408" "ENSMUSG00000099542" "ENSMUSG00000100107"
[11380] "ENSMUSG00000100051" "ENSMUSG00000099977" "ENSMUSG00000099415"
[11383

[11686] "ENSMUSG00000100712" "ENSMUSG00000099520" "ENSMUSG00000101119"
[11689] "ENSMUSG00000100520" "ENSMUSG00000102107" "ENSMUSG00000101057"
[11692] "ENSMUSG00000101125" "ENSMUSG00000101887" "ENSMUSG00000100521"
[11695] "ENSMUSG00000101790" "ENSMUSG00000100452" "ENSMUSG00000101403"
[11698] "ENSMUSG00000101830" "ENSMUSG00000100322" "ENSMUSG00000099350"
[11701] "ENSMUSG00000100329" "ENSMUSG00000099940" "ENSMUSG00000101766"
[11704] "ENSMUSG00000101530" "ENSMUSG00000100676" "ENSMUSG00000101539"
[11707] "ENSMUSG00000100353" "ENSMUSG00000101022" "ENSMUSG00000100165"
[11710] "ENSMUSG00000100077" "ENSMUSG00000101569" "ENSMUSG00000101184"
[11713] "ENSMUSG00000099386" "ENSMUSG00000095103" "ENSMUSG00000095785"
[11716] "ENSMUSG00000101982" "ENSMUSG00000099487" "ENSMUSG00000101507"
[11719] "ENSMUSG00000093993" "ENSMUSG00000100956" "ENSMUSG00000100942"
[11722] "ENSMUSG00000100333" "ENSMUSG00000100126" "ENSMUSG00000101679"
[11725] "ENSMUSG00000099434" "ENSMUSG00000101177" "ENSMUSG00000100993"
[11728

[12031] "ENSMUSG00000100317" "ENSMUSG00000101329" "ENSMUSG00000100532"
[12034] "ENSMUSG00000099374" "ENSMUSG00000100882" "ENSMUSG00000101772"
[12037] "ENSMUSG00000099667" "ENSMUSG00000099925" "ENSMUSG00000101582"
[12040] "ENSMUSG00000101273" "ENSMUSG00000101508" "ENSMUSG00000100339"
[12043] "ENSMUSG00000099546" "ENSMUSG00000101884" "ENSMUSG00000100906"
[12046] "ENSMUSG00000100092" "ENSMUSG00000100284" "ENSMUSG00000096523"
[12049] "ENSMUSG00000100986" "ENSMUSG00000101718" "ENSMUSG00000094454"
[12052] "ENSMUSG00000099736" "ENSMUSG00000101657" "ENSMUSG00000100740"
[12055] "ENSMUSG00000100718" "ENSMUSG00000100407" "ENSMUSG00000100946"
[12058] "ENSMUSG00000101851" "ENSMUSG00000100531" "ENSMUSG00000099483"
[12061] "ENSMUSG00000101742" "ENSMUSG00000100024" "ENSMUSG00000094660"
[12064] "ENSMUSG00000101052" "ENSMUSG00000100881" "ENSMUSG00000101160"
[12067] "ENSMUSG00000099840" "ENSMUSG00000100951" "ENSMUSG00000101068"
[12070] "ENSMUSG00000101405" "ENSMUSG00000099633" "ENSMUSG00000102079"
[12073

In [147]:
%%R
grep('ENSMUSG00000032315', ensembl2rxns.df$V1)

[1] 913917 913918 913919 913920 913921 913922 913923


In [148]:
%%R
reactome_ensembl_ids <- intersect(ensembl2rxns.df$V1,ensembl_wo_ids)
print(reactome_ensembl_ids)

   [1] "ENSMUSG00000000001" "ENSMUSG00000000028" "ENSMUSG00000000049"
   [4] "ENSMUSG00000000058" "ENSMUSG00000000085" "ENSMUSG00000000088"
   [7] "ENSMUSG00000000120" "ENSMUSG00000000126" "ENSMUSG00000000127"
  [10] "ENSMUSG00000000142" "ENSMUSG00000000149" "ENSMUSG00000000154"
  [13] "ENSMUSG00000000168" "ENSMUSG00000000171" "ENSMUSG00000000182"
  [16] "ENSMUSG00000000183" "ENSMUSG00000000184" "ENSMUSG00000000197"
  [19] "ENSMUSG00000000214" "ENSMUSG00000000215" "ENSMUSG00000000216"
  [22] "ENSMUSG00000000223" "ENSMUSG00000000247" "ENSMUSG00000000253"
  [25] "ENSMUSG00000000263" "ENSMUSG00000000275" "ENSMUSG00000000276"
  [28] "ENSMUSG00000000290" "ENSMUSG00000000296" "ENSMUSG00000000301"
  [31] "ENSMUSG00000000303" "ENSMUSG00000000305" "ENSMUSG00000000308"
  [34] "ENSMUSG00000000320" "ENSMUSG00000000326" "ENSMUSG00000000340"
  [37] "ENSMUSG00000000374" "ENSMUSG00000000394" "ENSMUSG00000000399"
  [40] "ENSMUSG00000000409" "ENSMUSG00000000420" "ENSMUSG00000000435"
  [43] "ENSMUSG00000

 [352] "ENSMUSG00000003541" "ENSMUSG00000003546" "ENSMUSG00000003549"
 [355] "ENSMUSG00000003555" "ENSMUSG00000003559" "ENSMUSG00000003573"
 [358] "ENSMUSG00000003604" "ENSMUSG00000003617" "ENSMUSG00000003623"
 [361] "ENSMUSG00000003644" "ENSMUSG00000003660" "ENSMUSG00000003665"
 [364] "ENSMUSG00000003731" "ENSMUSG00000003746" "ENSMUSG00000003752"
 [367] "ENSMUSG00000003779" "ENSMUSG00000003809" "ENSMUSG00000003812"
 [370] "ENSMUSG00000003813" "ENSMUSG00000003814" "ENSMUSG00000003824"
 [373] "ENSMUSG00000003848" "ENSMUSG00000003849" "ENSMUSG00000003863"
 [376] "ENSMUSG00000003865" "ENSMUSG00000003872" "ENSMUSG00000003873"
 [379] "ENSMUSG00000003882" "ENSMUSG00000003923" "ENSMUSG00000003934"
 [382] "ENSMUSG00000003970" "ENSMUSG00000003974" "ENSMUSG00000004018"
 [385] "ENSMUSG00000004032" "ENSMUSG00000004035" "ENSMUSG00000004038"
 [388] "ENSMUSG00000004040" "ENSMUSG00000004043" "ENSMUSG00000004044"
 [391] "ENSMUSG00000004054" "ENSMUSG00000004056" "ENSMUSG00000004070"
 [394] "ENSMUSG00000

 [703] "ENSMUSG00000008373" "ENSMUSG00000008435" "ENSMUSG00000008438"
 [706] "ENSMUSG00000008461" "ENSMUSG00000008475" "ENSMUSG00000008496"
 [709] "ENSMUSG00000008540" "ENSMUSG00000008590" "ENSMUSG00000008601"
 [712] "ENSMUSG00000008658" "ENSMUSG00000008668" "ENSMUSG00000008682"
 [715] "ENSMUSG00000008683" "ENSMUSG00000008690" "ENSMUSG00000008730"
 [718] "ENSMUSG00000008763" "ENSMUSG00000008845" "ENSMUSG00000008855"
 [721] "ENSMUSG00000008859" "ENSMUSG00000008892" "ENSMUSG00000008932"
 [724] "ENSMUSG00000008976" "ENSMUSG00000008999" "ENSMUSG00000009013"
 [727] "ENSMUSG00000009030" "ENSMUSG00000009073" "ENSMUSG00000009076"
 [730] "ENSMUSG00000009090" "ENSMUSG00000009092" "ENSMUSG00000009108"
 [733] "ENSMUSG00000009246" "ENSMUSG00000009281" "ENSMUSG00000009292"
 [736] "ENSMUSG00000009293" "ENSMUSG00000009350" "ENSMUSG00000009356"
 [739] "ENSMUSG00000009376" "ENSMUSG00000009394" "ENSMUSG00000009406"
 [742] "ENSMUSG00000009470" "ENSMUSG00000009487" "ENSMUSG00000009535"
 [745] "ENSMUSG00000

[1054] "ENSMUSG00000017718" "ENSMUSG00000017721" "ENSMUSG00000017723"
[1057] "ENSMUSG00000017724" "ENSMUSG00000017733" "ENSMUSG00000017737"
[1060] "ENSMUSG00000017740" "ENSMUSG00000017747" "ENSMUSG00000017754"
[1063] "ENSMUSG00000017756" "ENSMUSG00000017760" "ENSMUSG00000017765"
[1066] "ENSMUSG00000017774" "ENSMUSG00000017776" "ENSMUSG00000017778"
[1069] "ENSMUSG00000017781" "ENSMUSG00000017831" "ENSMUSG00000017837"
[1072] "ENSMUSG00000017843" "ENSMUSG00000017858" "ENSMUSG00000017861"
[1075] "ENSMUSG00000017868" "ENSMUSG00000017897" "ENSMUSG00000017929"
[1078] "ENSMUSG00000017950" "ENSMUSG00000017969" "ENSMUSG00000018001"
[1081] "ENSMUSG00000018008" "ENSMUSG00000018012" "ENSMUSG00000018040"
[1084] "ENSMUSG00000018042" "ENSMUSG00000018068" "ENSMUSG00000018102"
[1087] "ENSMUSG00000018126" "ENSMUSG00000018143" "ENSMUSG00000018160"
[1090] "ENSMUSG00000018166" "ENSMUSG00000018167" "ENSMUSG00000018169"
[1093] "ENSMUSG00000018189" "ENSMUSG00000018209" "ENSMUSG00000018263"
[1096] "ENSMUSG00000

[1405] "ENSMUSG00000020308" "ENSMUSG00000020309" "ENSMUSG00000020310"
[1408] "ENSMUSG00000020311" "ENSMUSG00000020312" "ENSMUSG00000020315"
[1411] "ENSMUSG00000020321" "ENSMUSG00000020325" "ENSMUSG00000020326"
[1414] "ENSMUSG00000020327" "ENSMUSG00000020329" "ENSMUSG00000020330"
[1417] "ENSMUSG00000020331" "ENSMUSG00000020333" "ENSMUSG00000020334"
[1420] "ENSMUSG00000020335" "ENSMUSG00000020340" "ENSMUSG00000020346"
[1423] "ENSMUSG00000020349" "ENSMUSG00000020357" "ENSMUSG00000020359"
[1426] "ENSMUSG00000020361" "ENSMUSG00000020362" "ENSMUSG00000020363"
[1429] "ENSMUSG00000020364" "ENSMUSG00000020366" "ENSMUSG00000020368"
[1432] "ENSMUSG00000020372" "ENSMUSG00000020375" "ENSMUSG00000020376"
[1435] "ENSMUSG00000020377" "ENSMUSG00000020380" "ENSMUSG00000020383"
[1438] "ENSMUSG00000020386" "ENSMUSG00000020387" "ENSMUSG00000020390"
[1441] "ENSMUSG00000020393" "ENSMUSG00000020395" "ENSMUSG00000020397"
[1444] "ENSMUSG00000020399" "ENSMUSG00000020400" "ENSMUSG00000020402"
[1447] "ENSMUSG00000

...

[8086] "ENSMUSG00000060534" "ENSMUSG00000060538" "ENSMUSG00000060550"
[8089] "ENSMUSG00000060572" "ENSMUSG00000060586" "ENSMUSG00000060591"
[8092] "ENSMUSG00000060600" "ENSMUSG00000060601" "ENSMUSG00000060630"
[8095] "ENSMUSG00000060636" "ENSMUSG00000060639" "ENSMUSG00000060671"
[8098] "ENSMUSG00000060675" "ENSMUSG00000060678" "ENSMUSG00000060679"
[8101] "ENSMUSG00000060681" "ENSMUSG00000060701" "ENSMUSG00000060708"
[8104] "ENSMUSG00000060733" "ENSMUSG00000060735" "ENSMUSG00000060743"
[8107] "ENSMUSG00000060780" "ENSMUSG00000060791" "ENSMUSG00000060798"
[8110] "ENSMUSG00000060802" "ENSMUSG00000060803" "ENSMUSG00000060807"
[8113] "ENSMUSG00000060860" "ENSMUSG00000060882" "ENSMUSG00000060904"
[8116] "ENSMUSG00000060938" "ENSMUSG00000060961" "ENSMUSG00000060969"
[8119] "ENSMUSG00000060981" "ENSMUSG00000060988" "ENSMUSG00000060992"
[8122] "ENSMUSG00000061048" "ENSMUSG00000061080" "ENSMUSG00000061086"
[8125] "ENSMUSG00000061099" "ENSMUSG00000061119" "ENSMUSG00000061126"
[8128] "ENSMUSG00000

[8437] "ENSMUSG00000066880" "ENSMUSG00000066892" "ENSMUSG00000066900"
[8440] "ENSMUSG00000066979" "ENSMUSG00000067081" "ENSMUSG00000067144"
[8443] "ENSMUSG00000067148" "ENSMUSG00000067149" "ENSMUSG00000067158"
[8446] "ENSMUSG00000067194" "ENSMUSG00000067199" "ENSMUSG00000067201"
[8449] "ENSMUSG00000067212" "ENSMUSG00000067219" "ENSMUSG00000067220"
[8452] "ENSMUSG00000067229" "ENSMUSG00000067231" "ENSMUSG00000067235"
[8455] "ENSMUSG00000067242" "ENSMUSG00000067274" "ENSMUSG00000067276"
[8458] "ENSMUSG00000067279" "ENSMUSG00000067288" "ENSMUSG00000067297"
[8461] "ENSMUSG00000067336" "ENSMUSG00000067338" "ENSMUSG00000067341"
[8464] "ENSMUSG00000067370" "ENSMUSG00000067424" "ENSMUSG00000067455"
[8467] "ENSMUSG00000067567" "ENSMUSG00000067586" "ENSMUSG00000067594"
[8470] "ENSMUSG00000067596" "ENSMUSG00000067597" "ENSMUSG00000067613"
[8473] "ENSMUSG00000067614" "ENSMUSG00000067615" "ENSMUSG00000067629"
[8476] "ENSMUSG00000067702" "ENSMUSG00000067713" "ENSMUSG00000067714"
[8479] "ENSMUSG00000

[8788] "ENSMUSG00000074657" "ENSMUSG00000074665" "ENSMUSG00000074673"
[8791] "ENSMUSG00000074677" "ENSMUSG00000074678" "ENSMUSG00000074695"
[8794] "ENSMUSG00000074698" "ENSMUSG00000074715" "ENSMUSG00000074731"
[8797] "ENSMUSG00000074733" "ENSMUSG00000074734" "ENSMUSG00000074743"
[8800] "ENSMUSG00000074768" "ENSMUSG00000074781" "ENSMUSG00000074785"
[8803] "ENSMUSG00000074793" "ENSMUSG00000074797" "ENSMUSG00000074824"
[8806] "ENSMUSG00000074852" "ENSMUSG00000074865" "ENSMUSG00000074867"
[8809] "ENSMUSG00000074886" "ENSMUSG00000074892" "ENSMUSG00000074899"
[8812] "ENSMUSG00000074916" "ENSMUSG00000074923" "ENSMUSG00000074934"
[8815] "ENSMUSG00000074939" "ENSMUSG00000074968" "ENSMUSG00000074991"
[8818] "ENSMUSG00000075000" "ENSMUSG00000075031" "ENSMUSG00000075122"
[8821] "ENSMUSG00000075140" "ENSMUSG00000075164" "ENSMUSG00000075196"
[8824] "ENSMUSG00000075202" "ENSMUSG00000075232" "ENSMUSG00000075266"
[8827] "ENSMUSG00000075267" "ENSMUSG00000075270" "ENSMUSG00000075271"
[8830] "ENSMUSG00000

[9139] "ENSMUSG00000094870" "ENSMUSG00000094891" "ENSMUSG00000094942"
[9142] "ENSMUSG00000094951" "ENSMUSG00000095028" "ENSMUSG00000095079"
[9145] "ENSMUSG00000095105" "ENSMUSG00000095117" "ENSMUSG00000095143"
[9148] "ENSMUSG00000095170" "ENSMUSG00000095210" "ENSMUSG00000095217"
[9151] "ENSMUSG00000095241" "ENSMUSG00000095253" "ENSMUSG00000095285"
[9154] "ENSMUSG00000095325" "ENSMUSG00000095388" "ENSMUSG00000095429"
[9157] "ENSMUSG00000095432" "ENSMUSG00000095463" "ENSMUSG00000095497"
[9160] "ENSMUSG00000095567" "ENSMUSG00000095571" "ENSMUSG00000095612"
[9163] "ENSMUSG00000095675" "ENSMUSG00000095677" "ENSMUSG00000095687"
[9166] "ENSMUSG00000095737" "ENSMUSG00000095788" "ENSMUSG00000095896"
[9169] "ENSMUSG00000095909" "ENSMUSG00000095990" "ENSMUSG00000096010"
[9172] "ENSMUSG00000096014" "ENSMUSG00000096054" "ENSMUSG00000096074"
[9175] "ENSMUSG00000096145" "ENSMUSG00000096146" "ENSMUSG00000096169"
[9178] "ENSMUSG00000096210" "ENSMUSG00000096255" "ENSMUSG00000096336"
[9181] "ENSMUSG00000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [149]:
%%R
grep('ENSMUSG00000032315', reactome_ensembl_ids)

[1] 5084


In [150]:
%%R
saveRDS(reactome_ensembl_ids,file=paste(OUT_DIR,"reactome_ensembl_ids030.Rds",sep=""))

In [151]:
%%R
print(grep('ENSMUSG00000032315', rownames(tcdd.df)))

[1] 40052


In [152]:
%%R
tcdd.df <- tcdd.df[reactome_ensembl_ids,]
print(tcdd.df)

                         V1       V2       V3       V4       V5       V6
ENSMUSG00000000001    54414    55963    71636    56315    55499    42459
ENSMUSG00000000028      483      371      785      445      508     3224
ENSMUSG00000000049   867859   869565  1380723  1012617  1016838   433465
ENSMUSG00000000058     1200     1446     1997     1048     1967     1679
ENSMUSG00000000085     5970     5071     7537     5648     4328     8975
ENSMUSG00000000088   102144   104159   135652    84658    82419   232996
ENSMUSG00000000120     3593     3842     5318     2860     2708     1844
ENSMUSG00000000126      299      599      300      649      500       50
ENSMUSG00000000127     3431     2944     5020     3461     4270     1780
ENSMUSG00000000142     3978     3978     5202     6884     5789     4056
ENSMUSG00000000149    63577    67422    87839    70522    62028    54098
ENSMUSG00000000154    46678    50051    69606    53885    54402    84566
ENSMUSG00000000168    30504    31531    52988    38

ENSMUSG00000001123   264484   265650   387854   298801   304169   160675
ENSMUSG00000001127    39275    40798    60205    46522    49501    48099
ENSMUSG00000001128    11737    12898    11458    10088    10022    35870
ENSMUSG00000001131       50       50        0        0      243     6774
ENSMUSG00000001143     8646     7832    13080     9712     8308     5796
ENSMUSG00000001155   353045   344441   499448   338014   330591   106861
ENSMUSG00000001158     5949     5270     9361     5246     5195    16059
ENSMUSG00000001173     3032     3171     6074     3164     4350     2684
ENSMUSG00000001175   133974   138059   191807   151772   155612   148933
ENSMUSG00000001211   111979   113748   151358   114714   112058   256044
ENSMUSG00000001225        0       49        0        0        0        0
ENSMUSG00000001228      350      500      550      196      149     2500
ENSMUSG00000001240    39713    37468    64969    33871    36522    87278
ENSMUSG00000001247   112785   115404   136978   108

ENSMUSG00000002289    12629    11667    31828    11101    12772    81508
ENSMUSG00000002297     1260     1519     1381      945      788     7843
ENSMUSG00000002307     7276     7464    10207     8225     8229     9752
ENSMUSG00000002308     2714     2680     2652     3314     2534     2731
ENSMUSG00000002324     1077      788     2328      448      648     1421
ENSMUSG00000002325    20146    21517    27420    24704    26582     7705
ENSMUSG00000002326     8550     8826    10024     7821     8404    24704
ENSMUSG00000002346    64896    67271    82118    63564    64902    21257
ENSMUSG00000002365    20649    21494    33607    24690    25591    22587
ENSMUSG00000002379    27330    24714    33813    24578    22873    85135
ENSMUSG00000002393    89139    89209    87844    80151    79453   114516
ENSMUSG00000002395    21672    22650    32645    22265    23828    51831
ENSMUSG00000002413    11226    11339    17190    12146    13883     4674
ENSMUSG00000002416    26716    27472    38286    22

ENSMUSG00000003435    46892    46870    45511    42232    41638    45477
ENSMUSG00000003436        0        0        0        0        0        0
ENSMUSG00000003437    18312    16939    24629    16270    13970    32391
ENSMUSG00000003444     2291     1597     3399     2106     2839     6440
ENSMUSG00000003452      100       99        0        0      100      250
ENSMUSG00000003458    23564    23364    28610    27214    26430    25314
ENSMUSG00000003464    55268    60757    80567    62438    60499    79523
ENSMUSG00000003476      150      350      200        0      250        0
ENSMUSG00000003477   440678   455643   943867   528577   518746     2160
ENSMUSG00000003484      337      833      973      526      436     5049
ENSMUSG00000003500     1178     1842     2002     1676     1602     7077
ENSMUSG00000003505        0        0        0        0        0        0
ENSMUSG00000003518    22042    22977    28494    20573    20396    39669
ENSMUSG00000003526   143958   144430   204413   146

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

ENSMUSG00000003644    5884    5807    5846    4306    8906    7458   21587
ENSMUSG00000003660   35591   35885   36448   35265   44738   44867  108444
ENSMUSG00000003665       0       0       0       0       0       0     148
ENSMUSG00000003731   26195   26364   24854   27782   42110   40251   98026
ENSMUSG00000003746  107760  110077  123633  126164  140065  144720  268016
ENSMUSG00000003752    3196    3363    2181    1843    3947    4074    9880
ENSMUSG00000003779    1914    1674    2168    2117    2418    2323    7559
ENSMUSG00000003809  272105  270669  268053  259682  361003  360344  284721
ENSMUSG00000003812   11162   10484    8250    8097   12279   11030   40793
ENSMUSG00000003813   24312   22163   19819   19097   25340   24513   59839
ENSMUSG00000003814  220920  223503  191206  203247  251302  243363  364868
ENSMUSG00000003824    2785    2411    2190    2512    2971    3688    3107
ENSMUSG00000003848    4017    4519    4576    4754    6524    6065   19368
ENSMUSG00000003849    554

ENSMUSG00000000214       0       0       0       0       0
ENSMUSG00000000215       0       0       0       0       0
ENSMUSG00000000216       0       0       0       0       0
ENSMUSG00000000223     550       0      28     345     448
ENSMUSG00000000247    2988    1194    1142    3225    3839
ENSMUSG00000000253    6251    2047    1898    4303    4300
ENSMUSG00000000263       0       0       0       0       0
ENSMUSG00000000275   95414   62142   60713   76661   75677
ENSMUSG00000000276   11596    4839    5486    5286    5300
ENSMUSG00000000290   41716   19830   18357   43155   42066
ENSMUSG00000000296    5943    3918    3728    4692    3629
ENSMUSG00000000301  101900   57244   58196  104949  106135
ENSMUSG00000000303   94435   47833   44051   69300   68820
ENSMUSG00000000305    6186    1635    1343    2985    2798
ENSMUSG00000000308     800      50       0      45     200
ENSMUSG00000000320    2373    1233    1421    2529    1505
ENSMUSG00000000326  410044  260125  246498  454017  4569

ENSMUSG00000001542   72496   44258   40175   61189   63275
ENSMUSG00000001552   33459   14808   14673   30965   32933
ENSMUSG00000001622      50       0       0     100     150
ENSMUSG00000001632   26919   15382   13311   26399   27596
ENSMUSG00000001642    6833    2103    2315    5996    5695
ENSMUSG00000001656       0       0       0       0       0
ENSMUSG00000001657      46       0       0       0       0
ENSMUSG00000001663   24927   24204   24161   33189   34137
ENSMUSG00000001670 2080209 1390350 1357794 1935941 1943572
ENSMUSG00000001729   97085   50922   48100   81959   86774
ENSMUSG00000001741    8860    5386    4277    9956    9237
ENSMUSG00000001750   94249   41026   37481  110190  107779
ENSMUSG00000001751   20095   10848   10460   14015   14755
ENSMUSG00000001755  105855   98162   93732  107710  110164
ENSMUSG00000001761   11689    4830    4717    9672    9332
ENSMUSG00000001767   21600   10333   10222   13695   14186
ENSMUSG00000001768   27145   13263   11785   21950   202

ENSMUSG00000003051    2567     243     155     792     650
ENSMUSG00000003053   81739  461984  450342  390967  394583
ENSMUSG00000003062   12803    7427    6373    5833    6931
ENSMUSG00000003068   56056   34556   33688   52640   54090
ENSMUSG00000003070      49     150      71     100     298
ENSMUSG00000003072  131195   71966   71087  186285  184792
ENSMUSG00000003099   36415   23277   19862   32968   33914
ENSMUSG00000003119   37386   21973   22098   37793   38388
ENSMUSG00000003131   69627   31765   31822   36139   37046
ENSMUSG00000003135   18756   10216   10119   13625   13316
ENSMUSG00000003153    1394    1349    1346    1485    1522
ENSMUSG00000003161   37421   15354   14411   19549   19592
ENSMUSG00000003184   40322   24258   22589   40469   43218
ENSMUSG00000003200   22212   13875   12594   24517   27144
ENSMUSG00000003206    1374    1037    1080    3235    2340
ENSMUSG00000003208    4069    2424    2366    4266    3320
ENSMUSG00000003226  255697  138322  134855  118478  1105

ENSMUSG00000004610  258718  159597  154911  138059  138476
ENSMUSG00000004626   68369   32371   31932   54294   56703
ENSMUSG00000004630     703     269     751     443     613
ENSMUSG00000004633    3604    6973    6569    6701    6275
ENSMUSG00000004637   17653    9117    9238   15698   14862
ENSMUSG00000004642   18295    8039    6608   11204   10869
ENSMUSG00000004651       0       0       0       0       0
ENSMUSG00000004654       0       0      50      50      50
ENSMUSG00000004655  398871  194121  191690  251519  251045
ENSMUSG00000004665   18649    6306    5001   17309   16933
ENSMUSG00000004667   27144   15922   13938   24731   25008
ENSMUSG00000004668      50       0      50      50       0
ENSMUSG00000004677   74839   29439   28972   55938   56123
ENSMUSG00000004709    2745     650     741    2045    2146
ENSMUSG00000004730   38863   21729   23131   41308   41512
ENSMUSG00000004768    8621    4464    4428    6488    4316
ENSMUSG00000004771   53691   24280   23543   30535   322

In [153]:
%%R
grep('ENSMUSG00000032315', rownames(tcdd.df))

[1] 5084


In [154]:
%%R
saveRDS(tcdd.df,file=paste(OUT_DIR,"tcdd_df030.Rds",sep=""))

In [155]:
%%R
saveRDS(deleted_data, file=paste(OUT_DIR, "format_name_deleted_data030.Rds", sep=""))